In [1]:
%matplotlib inline
import os, sys, numpy as np, pandas as pd, tensorflow as tf, re, codecs, seaborn as sns, json, time, csv, datetime as dt
import pickle, collections, random, math, numbers, scipy.sparse as sp, matplotlib.pyplot as plt, scipy.sparse as sp

from pprint import pprint
from tensorflow.contrib.training.python.training.hparam import HParams

def reload(mName):
    import importlib
    if mName in sys.modules:
        del sys.modules[mName]
    return importlib.import_module(mName)


from collections import deque, defaultdict, OrderedDict
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, minmax_scale
from matplotlib import pyplot as plt
plt.style.use('ggplot')

# classpath
ctx = os.path.abspath('..').replace('\\', '/')
cps = [ctx]
_ = [sys.path.insert(0, cp) for cp in cps if cp not in sys.path]

# data path
datapath = '/'.join([ctx, 'data'])

seed = 88
utils = reload('trainer.utils.utils')
np.set_printoptions(precision=4, suppress=True, linewidth=100)
np.random.seed(seed)

D:\Python\Anaconda3\envs\py3_5\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Simple Data Preprocess

In [55]:
import datetime as dt

ratings = pd.read_csv("{}/ml-latest-small/ratings.csv".format(datapath))
ratings['timestamp'] = ratings.timestamp.map(dt.datetime.fromtimestamp).map(str)
ratings['ori_rating'] = ratings['rating']
ratings['rating'] = (ratings.rating >= 4).astype(int)
tr, te = utils.split_by_ratio(ratings)

movies = pd.read_csv("{}/ml-latest-small/movies.csv".format(datapath))
avg_rt = ratings.groupby("movieId", as_index=False).ori_rating.mean().rename(index=str, columns={'ori_rating': 'avg_rating'})
movies = movies.merge(avg_rt, how='left', on='movieId')
# movies.avg_rating.fillna(ratings.rating.mean())
movies["year"] = movies.title.str.findall("\(\s*(\d+)\s*\)").map(lambda lst: int(lst[-1]) if len(lst) else None)
# movies["year"] = minmax_scale(movies.year.fillna(movies.year.median()))

In [ ]:
def preprocess(data, movie_trans, train_hist=None, is_train=True):
    queue = []
    data = data.merge(movie_trans, how="left", on="movieId")
    columns=["user_id", "query_movie_ids",
             "genres", "avg_rating", "year", "candidate_movie_id",
             "timestamp",
             "rating"]
    
    list2str = lambda lst: ','.join(map(str, lst))
    for u, df in data.groupby("userId"):
        df = df.sort_values("rating", ascending=False)
        if not is_train:
            user_movies_hist = train_hist.query("userId == {}".format(u)).movieId
        for i, (_, r) in enumerate(df.iterrows()):
            if is_train:
                query_hist = df.movieId[:i].tolist() + df.movieId[i + 1:].tolist()
                query_hist = list2str(query_hist)
                queue.append([int(r.userId), query_hist, r.genres, r.avg_rating, r.year, int(r.movieId), r.timestamp, r.rating])
            else:
                tr_hist = set(user_movies_hist.tolist())
                query_hist = list(tr_hist - set([int(r.movieId)]))
                query_hist = list2str(query_hist)
                queue.append([int(r.userId), query_hist, r.genres, r.avg_rating, r.year, int(r.movieId), r.timestamp, r.rating])
    return pd.DataFrame(queue, columns=columns)
    
tr_merged = preprocess(tr, movies)
tr_merged.to_csv('./tr.raw.movielens.csv', index=False, header=None)

te_merged = preprocess(te, movies, tr, is_train=False)
te_merged.to_csv('./te.raw.movielens.csv', index=False, header=None)
# 合併成一個檔案
merged = pd.concat([tr_merged, te_merged], ignore_index=True)
merged.to_csv('./merged_movielens.csv', index=False, header=None)
merged.head()

<br/>
<br/>
<br/>
## Cmd Submit Training

In [ ]:
!cd D:/Python/notebook/recomm_prod && \
gcloud ml-engine jobs submit training recomm_movielens_15 \
    --job-dir gs://recomm-job/foo/model \
    --runtime-version 1.4 \
    --module-name trainer.ctrl \
    --package-path trainer \
    --region asia-east1 \
    --config config.yaml \
    -- \
    --method train \
    --conf-path gs://recomm-job/foo/data/user_supplied/movielens.yaml

In [ ]:
!gcloud ml-engine jobs describe recomm_movielens_15

In [ ]:
!cd .. && python setup.py build

## Python Client API Transform Data

In [ ]:
from tensorflow.contrib.training.python.training.hparam import HParams

utils = reload('trainer.utils.utils')
env = reload('trainer.env')
reload('trainer.utils.flex')
reload('trainer.service')

ctrl = reload('trainer.ctrl').Ctrl.instance
hparam = HParams(conf_path='gs://movielens-foo/user_supplied/movielens.yaml')
hparam.add_hparam('is_local', False)
ctrl.gen_data(hparam)

## View Schema

In [4]:
from tensorflow.contrib.training.python.training.hparam import HParams

utils = reload('trainer.utils.utils')
env = reload('trainer.env')
reload('trainer.utils.flex')
reload('trainer.service')

ctrl = reload('trainer.ctrl').Ctrl.instance
params = {'conf_path': 'gs://movielens-foo/user_supplied/movielens.yaml'}
loader = ctrl.load_schema(params)

vars(loader.schema)

2018-03-06 15:53:38,551 - Loader - INFO [line:363] - try to unserialize from gs://recomm-job/foo-bar/movielens_recommendation/data/parsed.yaml


{'col_states_': OrderedDict([('query_movie_ids',
               CatgMapper(allow_null=True, default=None, is_multi=True,
                     name='query_movie_ids', sep=',', vocabs=None, vocabs_path=None)),
              ('genres',
               CatgMapper(allow_null=True, default=None, is_multi=True, name='genres',
                     sep='|', vocabs=None, vocabs_path=None)),
              ('avg_rating', NumericMapper(default=None, name='avg_rating')),
              ('year', NumericMapper(default=None, name='year')),
              ('candidate_movie_id',
               CatgMapper(allow_null=True, default=None, is_multi=False,
                     name='candidate_movie_id', sep=None, vocabs=None, vocabs_path=None)),
              ('rating',
               CatgMapper(allow_null=False, default=None, is_multi=False, name='rating',
                     sep=None, vocabs=None, vocabs_path=None))]),
 'conf_': {'columns': [{'id': 'user_id', 'm_dtype': 'catg'},
   {'id': 'query_movie_ids',
  

In [99]:
pad = tf.keras.preprocessing.sequence.pad_sequences
col = [[1, 2], [1, 2, 3], [1]]
maxlen = max(map(len, col))
pad(col, padding="post", maxlen=maxlen).tolist()


[[1, 2, 0], [1, 2, 3], [1, 0, 0]]

<br/>
<br/>
<br/>
## Python Client API Submit Training

In [112]:
utils = reload('trainer.utils.utils')
reload('trainer.env')
reload('trainer.utils.flex')
reload('trainer.service')

ctrl = reload('trainer.ctrl').Ctrl.instance
params = {'conf_path': 'gs://movielens-foo/user_supplied/movielens.yaml',
          'runtime_version': '1.4'}
ret = ctrl.train_submit(params)
job_id = ret.get('job_id')
print( ret.get('response') )

2018-03-07 15:09:58,977 - Ctrl - INFO [line:141] - foo-bar: gen_data take time 0:00:07.061886
jobId: movielens_recommendation_20180307150952344281
state: QUEUED
D:\Google\Cloud SDK\google-cloud-sdk\lib\googlecloudsdk\core\util\files.py:622: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  for chunk in iter(lambda: fp.read(4096), ''):
Job [movielens_recommendation_20180307150952344281] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe movielens_recommendation_20180307150952344281

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs movielens_recommendation_20180307150952344281



<br/>
<br/>
<br/>
## Describe Job States

In [118]:
from oauth2client.client import GoogleCredentials
from googleapiclient import discovery

env = reload('trainer.env')
utils = reload('trainer.utils.utils')
flex = reload('trainer.utils.flex')
reload('trainer.service')

ctrl = reload('trainer.ctrl').Ctrl.instance
params = {'conf_path': 'gs://movielens-foo/user_supplied/movielens.yaml', 'job_id': job_id}
ret = ctrl.describe(params)
ret['response']

2018-03-07 15:23:32,889 - googleapiclient.discovery - INFO [line:868] - URL being requested: GET https://ml.googleapis.com/v1/projects/training-recommendation-engine/jobs/movielens_recommendation_20180307150952344281?alt=json
2018-03-07 15:23:32,891 - oauth2client.transport - INFO [line:151] - Attempting refresh to obtain initial access_token
2018-03-07 15:23:32,932 - oauth2client.client - INFO [line:795] - Refreshing access_token
2018-03-07 15:23:34,148 - Ctrl - INFO [line:157] - foo-bar: describe take time 0:00:01.278954


{'createTime': '2018-03-07T07:09:58Z',
 'endTime': '2018-03-07T07:18:28Z',
 'jobId': 'movielens_recommendation_20180307150952344281',
 'startTime': '2018-03-07T07:11:43Z',
 'state': 'SUCCEEDED',
 'trainingInput': {'args': ['--train-steps',
   '1000',
   '--method',
   'train',
   '--conf-path',
   'gs://movielens-foo/user_supplied/movielens.yaml',
   '--job-id',
   'movielens_recommendation_20180307150952344281'],
  'jobDir': 'gs://recomm-job/foo-bar/movielens_recommendation/model',
  'packageUris': ['gs://recomm-job/foo-bar/movielens_recommendation/model/packages/2643cd1b31dcc17417a23c46e7784c9ca6294d77730e79cba505a8c2e2c0df78/trainer-0.1.tar.gz'],
  'pythonModule': 'trainer.ctrl',
  'pythonVersion': '3.5',
  'region': 'asia-east1',
  'runtimeVersion': '1.4'},
 'trainingOutput': {'consumedMLUnits': 0.1}}

## Deploy

In [92]:
from oauth2client.client import GoogleCredentials
from googleapiclient import discovery

utils = reload('trainer.utils.utils')
env = reload('trainer.env')
flex = reload('trainer.utils.flex')
reload('trainer.service')

ctrl = reload('trainer.ctrl').Ctrl.instance
params = {'conf_path': 'gs://movielens-foo/user_supplied/movielens.yaml'}
ret = ctrl.deploy(params)
ret

2018-03-07 17:06:51,543 - Service - INFO [line:98] - try to create model [foo_bar_movielens_recommendation] ...
2018-03-07 17:06:51,563 - googleapiclient.discovery - INFO [line:868] - URL being requested: POST https://ml.googleapis.com/v1/projects/training-recommendation-engine/models?alt=json
2018-03-07 17:06:51,564 - oauth2client.transport - INFO [line:151] - Attempting refresh to obtain initial access_token
2018-03-07 17:06:51,600 - oauth2client.client - INFO [line:795] - Refreshing access_token
2018-03-07 17:06:57,030 - Service - INFO [line:101] - try to clean old version ...
2018-03-07 17:06:57,040 - googleapiclient.discovery - INFO [line:868] - URL being requested: GET https://ml.googleapis.com/v1/projects/training-recommendation-engine/models/foo_bar_movielens_recommendation/versions?alt=json
2018-03-07 17:06:57,354 - googleapiclient.discovery - INFO [line:868] - URL being requested: POST https://ml.googleapis.com/v1/projects/training-recommendation-engine/models/foo_bar_moviele

{'err_cde': '00',
 'response': {'metadata': {'@type': 'type.googleapis.com/google.cloud.ml.v1.OperationMetadata',
   'createTime': '2018-03-07T09:06:59Z',
   'modelName': 'projects/training-recommendation-engine/models/foo_bar_movielens_recommendation',
   'operationType': 'CREATE_VERSION',
   'version': {'createTime': '2018-03-07T09:06:57Z',
    'deploymentUri': 'gs://recomm-job/foo-bar/movielens_recommendation/model/export/export_foo-bar/1520406885',
    'description': '[foo-bar] recommendation model',
    'name': 'projects/training-recommendation-engine/models/foo_bar_movielens_recommendation/versions/v20180307170657344373',
    'runtimeVersion': '1.4'}},
  'name': 'projects/training-recommendation-engine/operations/create_foo_bar_movielens_recommendation_v20180307170657344373-1520413617999'}}

## Get Information From Deployed Model

In [97]:
from oauth2client.client import GoogleCredentials
from googleapiclient import discovery

utils = reload('trainer.utils.utils')
env = reload('trainer.env')
flex = reload('trainer.utils.flex')
reload('trainer.service')

ctrl = reload('trainer.ctrl').Ctrl.instance
params = {'conf_path': 'gs://movielens-foo/user_supplied/movielens.yaml'}
ret = ctrl.model_info(params)
ret

2018-03-07 17:08:40,844 - googleapiclient.discovery - INFO [line:868] - URL being requested: GET https://ml.googleapis.com/v1/projects/training-recommendation-engine/models/foo_bar_movielens_recommendation/versions/v20180307170657344373?alt=json
2018-03-07 17:08:40,849 - oauth2client.transport - INFO [line:151] - Attempting refresh to obtain initial access_token
2018-03-07 17:08:40,882 - oauth2client.client - INFO [line:795] - Refreshing access_token


{'err_cde': '00',
 'response': {'createTime': '2018-03-07T09:06:57Z',
  'deploymentUri': 'gs://recomm-job/foo-bar/movielens_recommendation/model/export/export_foo-bar/1520406885',
  'description': '[foo-bar] recommendation model',
  'isDefault': True,
  'name': 'projects/training-recommendation-engine/models/foo_bar_movielens_recommendation/versions/v20180307170657344373',
  'runtimeVersion': '1.4',
  'state': 'READY'}}

## Restful predict

In [96]:
ratings = pd.read_csv("{}/ml-latest-small/ratings.csv".format(datapath))
ratings['timestamp'] = ratings.timestamp.map(dt.datetime.fromtimestamp).map(str)
ratings['ori_rating'] = ratings['rating']
ratings['rating'] = (ratings.rating >= 4).astype(int)

movies = pd.read_csv("{}/ml-latest-small/movies.csv".format(datapath))
avg_rt = ratings.groupby("movieId", as_index=False).ori_rating.mean().rename(index=str, columns={'ori_rating': 'avg_rating'})
movies = movies.merge(avg_rt, how='left', on='movieId')
movies["year"] = movies.title.str.findall("\(\s*(\d+)\s*\)").map(lambda lst: int(lst[-1]) if len(lst) else None)

ctrl = reload('trainer.ctrl').Ctrl.instance
params = {'conf_path': '../data/foo/user_supplied/movielens.yaml'}
loader = ctrl.load_schema(params)
merged = pd.read_csv('../data/foo/user_supplied/raws/merged_movielens.csv', names=loader.schema.raw_cols)
merged.head()

2018-03-07 17:08:28,342 - Loader - INFO [line:363] - try to unserialize from gs://recomm-job/foo-bar/movielens_recommendation/data/parsed.yaml


,user_id,query_movie_ids,genres,avg_rating,year,candidate_movie_id,timestamp,rating
0,1,"1953,2105,31,1029,1061,1129,1263,1287,1293,133...",Drama,4.260870,1989.0,1172,2009-12-14 10:53:25,1
1,1,"1172,2105,31,1029,1061,1129,1263,1287,1293,133...",Action|Crime|Thriller,4.021739,1971.0,1953,2009-12-14 10:53:11,1
2,1,"1172,1953,31,1029,1061,1129,1263,1287,1293,133...",Action|Adventure|Sci-Fi,3.478723,1982.0,2105,2009-12-14 10:52:19,1
3,1,"1172,1953,2105,1029,1061,1129,1263,1287,1293,1...",Drama,3.178571,1995.0,31,2009-12-14 10:52:24,0
4,1,"1172,1953,2105,31,1061,1129,1263,1287,1293,133...",Animation|Children|Drama|Musical,3.702381,1941.0,1029,2009-12-14 10:52:59,0


In [113]:
utils = reload('trainer.utils.utils')
env = reload('trainer.env')
flex = reload('trainer.utils.flex')
reload('trainer.service')

user_ids = (22,)
data = {
    'query_movie_ids': merged.query('user_id in {}'.format(user_ids)).groupby('user_id').query_movie_ids.max().tolist(),
}
items = movies.rename(index=str, columns={"movieId": "candidate_movie_id"}).drop('title', 1)
items.loc[:, 'candidate_movie_id'] = items.candidate_movie_id.astype(str)
items = items.to_dict('list')
data.update(items)

ctrl = reload('trainer.ctrl').Ctrl.instance
params = {'conf_path': 'gs://movielens-foo/user_supplied/movielens.yaml',
          'json_data': data}
ret = ctrl.predict(params)
ret.get('response')

2018-03-07 18:02:52,743 - Loader - INFO [line:363] - try to unserialize from gs://recomm-job/foo-bar/movielens_recommendation/data/parsed.yaml
2018-03-07 18:02:54,649 - Loader - INFO [line:451] - try to restful transform ... 
2018-03-07 18:02:57,089 - Ctrl - INFO [line:327] - foo-bar: predict take time 0:00:04.648373


'{\r\n  "error": "Prediction failed: Error during model execution: AbortionError(code=StatusCode.INVALID_ARGUMENT, details=\\"input size does not match signature\\")"\r\n}\r\n'

In [ ]:
ret.get('data_for_model')

In [85]:
"""
query_movie_ids
genres
avg_rating
year
candidate_movie_id
query_movie_ids_len
genres_len
"""
path = 'D:/Python/notebook/recomm_prod/repo/foo-bar/movielens_recommendation/1520406885'
with tf.Session() as sess:
    tf.saved_model.loader.load(sess, [tf.saved_model.tag_constants.SERVING], path)
    # graph = sess.graph
    # print(sess.run(graph.get_tensor_by_name('gmf/pred:0'), feed_dict={
    #     graph.get_tensor_by_name('query_movie_ids:0'): data_for_model.get('query_movie_ids'),
    #     graph.get_tensor_by_name('genres:0'): data_for_model.get('genres'),
    #     graph.get_tensor_by_name('avg_rating:0'): data_for_model.get('avg_rating'),
    #     graph.get_tensor_by_name('year:0'): data_for_model.get('year'),
    #     graph.get_tensor_by_name('candidate_movie_id:0'): data_for_model.get('candidate_movie_id'),
    #     graph.get_tensor_by_name('query_movie_ids_len:0'): data_for_model.get('query_movie_ids_len'),
    #     graph.get_tensor_by_name('genres_len:0'): data_for_model.get('genres_len')
    # }))
    for node in sess.graph.get_operations():
        shape = node.node_def.attr.get('shape')
        if shape is not None:
            print(node.name, shape)

INFO:tensorflow:Restoring parameters from b'D:/Python/notebook/recomm_prod/repo/foo-bar/movielens_recommendation/1520406885\\variables\\variables'
2018-03-07 16:58:16,000 - tensorflow - INFO [line:116] - Restoring parameters from b'D:/Python/notebook/recomm_prod/repo/foo-bar/movielens_recommendation/1520406885\\variables\\variables'
global_step shape {
}

query_movie_ids shape {
  dim {
    size: -1
  }
  dim {
    size: -1
  }
}

genres shape {
  dim {
    size: -1
  }
  dim {
    size: -1
  }
}

avg_rating shape {
  dim {
    size: -1
  }
}

year shape {
  dim {
    size: -1
  }
}

candidate_movie_id shape {
  dim {
    size: -1
  }
}

query_movie_ids_len shape {
  dim {
    size: -1
  }
}

genres_len shape {
  dim {
    size: -1
  }
}

init/b_global shape {
}

init/embedding/w_query_movie_ids shape {
  dim {
    size: 9125
  }
  dim {
    size: 16
  }
}

init/embedding/b_query_movie_ids shape {
  dim {
    size: 16
  }
}

init/embedding/w_candidate_movie_id shape {
  dim {
    size:

<br/>
<br/>
<br/>
## Local Transform

In [ ]:
from tensorflow.contrib.training.python.training.hparam import HParams

utils = reload('trainer.utils.utils')
env = reload('trainer.env')
flex = reload('trainer.utils.flex')
service = reload('trainer.service')

ctrl = reload('trainer.ctrl').Ctrl.instance
params = {'conf_path': '../data/foo/user_supplied/movielens.local.yaml', 'is_local': True}
ctrl.gen_data(params)

<br/>
<br/>
<br/>
## Local View Schema

In [ ]:
utils = reload('trainer.utils.utils')
env = reload('trainer.env')
flex = reload('trainer.utils.flex')
service = reload('trainer.service')

with flex.io('../repo/foo-bar/movielens_recommendation/data/parsed.yaml').as_reader() as f:
    schema = flex.Schema.unserialize(f.stream)
vars(schema)

<br/>
<br/>
<br/>
## Local Training

In [86]:
from tensorflow.contrib.training.python.training.hparam import HParams

utils = reload('trainer.utils.utils')
env = reload('trainer.env')
reload('trainer.reco_mf_dnn_est')
flex = reload('trainer.utils.flex')
reload('trainer.service')

ctrl = reload('trainer.ctrl').Ctrl.instance
params = {'conf_path': '../data/foo/user_supplied/movielens.local.yaml', 
          'is_local': True,
          'runtime_version': '1.4',
          'train_steps': 600}
ret = ctrl.train(params)

2018-03-07 14:16:22,970 - Ctrl - INFO [line:170] - received params: {'train_steps': 600, 'conf_path': '../data/foo/user_supplied/movielens.local.yaml', 'is_local': True, 'runtime_version': '1.4'}
2018-03-07 14:16:22,973 - Ctrl - INFO [line:176] - do local training
2018-03-07 14:16:23,007 - Ctrl - INFO [line:196] - foo-bar: try to unserialize D:\Python\notebook\recomm_prod\repo/foo-bar/movielens_recommendation/data/parsed.yaml
2018-03-07 14:16:25,442 - Service - INFO [line:41] - received params: {'n_batch': 128, 'job_dir': 'D:\\Python\\notebook\\recomm_prod\\repo/foo-bar/movielens_recommendation/model', 'model_id': 'movielens_recommendation', 'runtime_version': '1.4', 'train_steps': 600, 'train_file': 'D:\\Python\\notebook\\recomm_prod\\repo/foo-bar/movielens_recommendation/data/data.tr', 'repo': 'D:\\Python\\notebook\\recomm_prod\\repo/foo-bar/movielens_recommendation', 'dim': 16, 'export_name': 'export_foo-bar', 'eval_name': 'foo-bar', 'pid': 'foo-bar', 'conf_path': '../data/foo/user_

2018-03-07 14:16:51,096 - tensorflow - INFO [line:116] - Evaluation [230/233]
INFO:tensorflow:Evaluation [233/233]
2018-03-07 14:16:51,123 - tensorflow - INFO [line:116] - Evaluation [233/233]
INFO:tensorflow:Finished evaluation at 2018-03-07-06:16:51
2018-03-07 14:16:51,149 - tensorflow - INFO [line:116] - Finished evaluation at 2018-03-07-06:16:51
INFO:tensorflow:Saving dict for global step 549: auc = 0.79919815, global_step = 549, loss = 0.54601425
2018-03-07 14:16:51,151 - tensorflow - INFO [line:116] - Saving dict for global step 549: auc = 0.79919815, global_step = 549, loss = 0.54601425
2018-03-07 14:16:51,168 - BestScoreExporter - INFO [line:236] - clean export_path: D:\Python\notebook\recomm_prod\repo/foo-bar/movielens_recommendation/model\export\export_foo-bar
2018-03-07 14:16:51,175 - BestScoreExporter - INFO [line:241] - nice eval loss: 0.5460142493247986, export to pb
INFO:tensorflow:Calling model_fn.
2018-03-07 14:16:51,193 - tensorflow - INFO [line:116] - Calling model_f

In [84]:
model = ret.get('response')
vars(model)

{'ans': <tf.Tensor 'loss/strided_slice:0' shape=(?, 1) dtype=float32>,
 'auc': (<tf.Tensor 'metrics/auc/value:0' shape=() dtype=float32>,
  <tf.Tensor 'metrics/auc/update_op:0' shape=() dtype=float32>),
 'avg_rating': <tf.Tensor 'IteratorGetNext:0' shape=(?,) dtype=float32>,
 'b_candidate_movie_id': <tf.Variable 'init/embedding/b_candidate_movie_id:0' shape=(26,) dtype=float32_ref>,
 'b_global': <tf.Variable 'init/b_global:0' shape=() dtype=float32_ref>,
 'b_query_movie_ids': <tf.Variable 'init/embedding/b_query_movie_ids:0' shape=(16,) dtype=float32_ref>,
 'candidate_bias': <tf.Tensor 'item_encoding/MatMul:0' shape=(?, 1) dtype=float32>,
 'candidate_emb': <tf.Tensor 'item_encoding/embedding_lookup:0' shape=(?, 16) dtype=float32>,
 'candidate_movie_id': <tf.Tensor 'IteratorGetNext:1' shape=(?,) dtype=int32>,
 'emb_genres': <tf.Tensor 'item_encoding/Sum:0' shape=(?, 8) dtype=float32>,
 'emb_item': <tf.Tensor 'item_encoding/dense_2/Selu:0' shape=(?, 32) dtype=float32>,
 'emb_query': <tf.

## Test

In [4]:
from tensorflow.contrib.training.python.training.hparam import HParams

utils = reload('trainer.utils.utils')
env = reload('trainer.env')
reload('trainer.reco_mf_dnn_est')
reload('trainer.utils.flex')
reload('trainer.service')

ctrl = reload('trainer.ctrl').Ctrl.instance
params = {'conf_path': '../data/foo/user_supplied/movielens.local.yaml', 
          'raw_dir': 'gs://recomm-job/foo-bar'}
print(ctrl.test(params).get('response'))

{'export_path': 'gs://recomm-job/foo-bar/movielens_recommendation/model/export/export_foo-bar/1520326728', 'model_name': 'foo_bar_movielens_recommendation', 'version': 'v20180306181514380085', 'job_id': 'movielens_recommendation_20180306165443177431'}


### 更改GCS movielens.yaml

In [ ]:
import shutil
from google.cloud.storage.blob import Blob
from io import BytesIO

utils = reload('trainer.utils.utils')
flex = reload('trainer.utils.flex')
env = reload('trainer.env')

with flex.io('../data/foo/user_supplied/movielens.yaml') as r, \
    flex.io('gs://movielens-foo/user_supplied/movielens.yaml') as w:
    w.write(r.read())

# stream = BytesIO(open('../data/foo/user_supplied/movielens.yaml', mode='rb').read())
# utils.gcs_blob('gs://movielens-foo/user_supplied/movielens.yaml').upload_from_file(stream)

<br/>
<br/>
<br/>
## Python API Credential

In [ ]:
from oauth2client.client import GoogleCredentials
from googleapiclient import discovery
from google.cloud import storage

authpath = '../auth.json'
project = 'training-recommendation-engine'
cred = GoogleCredentials.from_stream(authpath)
svc = discovery.build('ml', 'v1', credentials=cred)

st_client = storage.Client.from_service_account_json(authpath)
bucket = st_client.get_bucket('recomm-job')

In [ ]:
from io import StringIO, BytesIO

blob = bucket.get_blob('user_supplied/raws/merged_movielens.csv')
sio = BytesIO()
blob.download_to_file(sio)

In [ ]:
cols = list(data.keys())
multi_cols = ('query_movie_ids', 'genres')

def trans(features):
    # features = OrderedDict(zip(cols, data))
    print( schema.col_states_['query_movie_ids'].transform( features['query_movie_ids'] ) )
    # for col in multi_cols:
    #     features[col] = tf.string_to_number(tf.string_split(features[col], ',').values, out_type=tf.int32)
    return features

def add_seq_cols(feat):
    for m_col in multi_cols:
        name = '{}_len'.format(m_col)
        feat[name] = tf.size(feat[m_col])
        cols.append(name)
    return feat

tf.reset_default_graph()
with tf.Graph().as_default():
    dataset = tf.data.Dataset.from_tensors(data)
    dataset = dataset.map(trans, num_parallel_calls=4)
    dataset = dataset.map(add_seq_cols, num_parallel_calls=4)
    print('cols', cols)
    dataset = dataset.repeat(1)
    dataset = dataset.padded_batch(5, OrderedDict(zip(cols, ([], [], [], [None], [], [None], [], []))))
    inputs = dataset.make_one_shot_iterator().get_next()
    with tf.train.MonitoredTrainingSession() as sess:
        while not sess.should_stop():
            _, = sess.run([inputs])
            # print( sess.run(inputs) )
            pass

<br/>
<br/>
<br/>
## Dataset

In [ ]:
def make_datasets(fpath_ary, schema, n_batch=128, n_epoch=1):
    def to_dense(sp):
        dense = tf.sparse_to_dense(sp.indices, sp.dense_shape, sp.values, '')
        return tf.reshape(tf.to_int32(tf.string_to_number(dense)), [-1])

    def to_sparse(dense):
        idx = tf.where(tf.not_equal(dense, 0))
        return tf.SparseTensor(indices=idx, dense_shape=dense.get_shape(), values=tf.gather_nd(dense, idx))

    def parse_csv(value):
        data = tf.decode_csv(value, record_defaults=defaults)
        features = OrderedDict(zip(cols, data))
        multi_cols = df_conf.query("{} == '{}' and {} == True".format(schema.M_DTYPE, schema.CATG, schema.IS_MULTI)).id.values
        for col in multi_cols:
            features[col] = tf.string_split([features[col]], ',')
            features[col] = to_dense(features[col])
            # features['{}_lens'.format(col)] = tf.size(features[col])
        return features 
    
    df_conf = schema.df_conf_.query('{}.notnull()'.format(schema.TYPE))
    cols = schema.cols
    defaults = []
    for _, r in df_conf.iterrows():
        if r[schema.M_DTYPE] == schema.CATG:
            defaults.append([''] if r[schema.IS_MULTI] else [0])
        else:
            defaults.append([])
    dataset = tf.data.TextLineDataset(fpath_ary)
    dataset = dataset.map(parse_csv, num_parallel_calls=4)
    has_multi = (df_conf[schema.M_DTYPE] == schema.CATG) & (df_conf[schema.IS_MULTI] == True)
    if sum(has_multi):
        multi_cols = df_conf[has_multi].id.values
        dataset = dataset.padded_batch(n_batch, OrderedDict( zip(cols, tuple([None] if e else [] for e in has_multi))) )
    else:
        dataset = dataset.batch(n_batch)
    dataset = dataset.shuffle(n_batch * 10, seed=seed).repeat(n_epoch)
    features = dataset.make_one_shot_iterator().get_next()
    return features, features.pop(schema.label[0])
                                
# tf.reset_default_graph()
with tf.Graph().as_default():
    inputs = make_datasets(['./movielens.tr'], loader.schema, n_batch=30)
    query_lens = tf.sequence_mask([1, 2, 3])
    ctx = []
    with tf.train.MonitoredTrainingSession() as sess:
        while not sess.should_stop():
            _, = sess.run([inputs])
            # print( sess.run(inputs) )
            pass

## Feature Columns with tf.feature_column.input_layer

In [ ]:
a = pd.Series(minmax_scale(np.random.normal(0, 1, size=1000)))
a.hist(bins=50)

In [ ]:
%%time
tf.reset_default_graph()
with tf.Graph().as_default():
    user_id = tf.feature_column.categorical_column_with_hash_bucket('user_id', hash_bucket_size=1000, dtype=tf.int32)
    user_id = tf.feature_column.embedding_column(user_id, dimension=8)
    avg_rating = tf.feature_column.numeric_column('avg_rating')
    columns = [user_id, avg_rating]
    
    def make_datasets(fpath_ary):
        cols = ['user_id', 'query_movie_ids', 'genres', 'avg_rating', 'year', 'candidate_movie_id', 'rating']
        defaults = [[0], [''], [''], [], [], [0], []]

        def parse_csv(value):
            data = tf.decode_csv(value, record_defaults=defaults)
            features = OrderedDict(zip(cols, data))
            # print(features)
            return features
        
        dataset = tf.data.TextLineDataset(fpath_ary)
        dataset = (dataset.map(parse_csv, num_parallel_calls=4)
                          .batch(3)
                          # .padded_batch(3, OrderedDict(zip(cols, ([], [None], [None], [], [], [], []))))
                          .shuffle(10, seed=seed)
                          .repeat(1)
                  )
        return dataset.make_one_shot_iterator().get_next()
    
    inputs = make_datasets(['./te_processed.batch.csv'])
    inputs = tf.feature_column.input_layer(inputs, columns)
    # features = tf.parse_example(serialized_example, features=tf.feature_column.make_parse_example_spec(columns))
    ctx = []
    with tf.train.MonitoredTrainingSession() as sess:
        while not sess.should_stop():
            print(sess.run(inputs))

### Make Example

In [ ]:
%%time
cols = ['user_id', 'query_movie_ids', 'genres', 'avg_rating', 'year', 'candidate_movie_id', 'rating']
is_multi = [False, True, True, False, False, False, False]
pd_dtypes = [int, str, str, float, float, int, float]
types = ['int64_list', 'int64_list', 'int64_list', 'float_list', 'float_list', 'int64_list', 'float_list']
tf_types = [tf.int64, tf.int64, tf.int64, tf.float32, tf.float32, tf.int64, tf.float32]
def persist_example(fpath, tfpath):
    with tf.python_io.TFRecordWriter(tfpath) as w:
        for chunk in pd.read_csv(fpath, names=cols, dtype=dict(zip(cols, pd_dtypes)), chunksize=1000):
            chunk['query_movie_ids'] = chunk.query_movie_ids.map(lambda r: map(int, r.split(',')))
            chunk['genres'] = chunk.genres.map(lambda r: map(int, r.split(',')))
            
            for idx, r in chunk.iterrows():
                ex = tf.train.Example()
                for multi, col, tpe in zip(is_multi, cols, types):
                    val = r[col]
                    # ex.features.feature[col].int64_list or float_list or bytes_list
                    feat_type = getattr(ex.features.feature[col], tpe)
                    # extend function for multivalent columns, otherwise append
                    append_or_extend = 'append' if not multi else 'extend'                    
                    getattr(feat_type.value, append_or_extend)(val)
                w.write(ex.SerializePartialToString())

persist_example('./te_processed.csv', './data.tfrecord')

In [ ]:
def decode_example(ser_example):
    # queue = tf.train.string_input_producer([fpath], num_epochs=1)
    # _, ser_example = tf.TFRecordReader().read(queue)
    # ser_example = tf.train.batch([ser_example], batch_size=10)
    ctx_features = {col: tf.FixedLenFeature([], tf_tpe)
                    for col, tf_tpe in zip(cols, tf_types) if col not in ('query_movie_ids', 'genres')}
    seq_features = {col: tf.FixedLenSequenceFeature([], tf_tpe) 
                    for col, tf_tpe in [('query_movie_ids', tf.int64), ('genres', tf.int64)]}
    context_dict, sequence_dict = tf.parse_single_sequence_example(ser_example, 
                                                                   context_features=ctx_features, 
                                                                   sequence_features=seq_features)
    # for col, tpe in zip(cols, tf_types):
    #     val = feature_dict[col]
    #     feature_dict[col] = tf.sparse_to_dense(val.indices, val.dense_shape, val.values, name=col)
    feature_dict = {}
    feature_dict.update(context_dict)
    feature_dict.update(sequence_dict)
    ret = OrderedDict()
    for c in cols:
        ret[c] = feature_dict[c]
    return tuple(ret.values())

tf.reset_default_graph()
with tf.Graph().as_default():
    dataset = tf.data.TFRecordDataset(['./data.tfrecord'])
    dataset = dataset.map(decode_example).padded_batch(10, padded_shapes=([], [None], [None], [], [], [], []))
    # dataset = dataset.batch(3)
    iters = dataset.make_one_shot_iterator()
    r = iters.get_next()
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.tables_initializer())
        print( sess.run(r) )

## Traditional parse_example
1. tf.train.Coordinator + tf.train.start_queue_runners

In [ ]:
from tensorflow.python.framework import sparse_tensor
import re

def to_sparse(dense):
    idx = tf.where(tf.not_equal(dense, 0))
    return tf.SparseTensor(idx, tf.gather_nd(dense, idx), dense.get_shape())

def make_example(val):
    example = tf.train.Example(features=tf.train.Features(
        feature = {
            'query_movie_ids': tf.train.Feature(int64_list=tf.train.Int64List(value=val)),
            'genres': tf.train.Feature(int64_list=tf.train.Int64List(value=val))
        }
    ))
    return example

tf.reset_default_graph()
with tf.Graph().as_default():
    
    filename = "tmp.tfrecords"
    if not os.path.exists(filename):
        # os.remove(filename)
        writer = tf.python_io.TFRecordWriter(filename)
        with writer:
            for idx, r in teProcessed.head().iterrows():
                for col in ('query_movie_ids', 'genres'):
                    val = list(map(int, re.split(',\s*', r[col])))
                    ex = make_example(val)
                    writer.write(ex.SerializeToString())

    reader = tf.TFRecordReader()
    filename_queue = tf.train.string_input_producer(["tmp.tfrecords"], num_epochs=1)
    _, serialized_example = reader.read(filename_queue)

    batch = tf.train.batch(tensors=[serialized_example], batch_size=1)
    features = {
        'query_movie_ids': tf.VarLenFeature(tf.int64),
        'genres': tf.VarLenFeature(tf.int64)
    }
    data = tf.parse_example(batch, features)
    query_movie_ids = data['query_movie_ids']
    embbedding = tf.Variable(tf.glorot_uniform_initializer()([9125]), dtype=tf.float32)
    print(query_movie_ids.dense_shape)
    # r = tf.layers.dense(query_movie_ids, 10)
    # emb_query = tf.nn.embedding_lookup_sparse([embbedding], query_movie_ids, None, combiner='sqrtn')
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        tf.local_variables_initializer().run()
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coord, sess=sess)
        try:
            print(sess.run(data))
            pass
        except tf.errors.OutOfRangeError as e:
            coord.request_stop(e)
        finally:
            coord.request_stop()
            coord.join(threads)
    

## Test

In [ ]:
tf.reset_default_graph()
with tf.Graph().as_default():
    labels = tf.constant(np.ones([10, 8]))
    pred = tf.concat([tf.Variable(tf.ones(shape=[1, 8]), trainable=False), tf.Variable(tf.truncated_normal([9, 8]))], 0)
    loss = tf.losses.mean_squared_error(predictions=pred, labels=labels)
    train_op = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        print(pred.eval())
        for i in range(1000):
            sess.run([train_op])
        print()
        print(pred.eval())

In [ ]:
tf.zeros